In [ ]:
import pandas as pd
from S1000_model import S1000_comp_model
from names import DICT_COMP_SIGNALKEY
import numpy as np
import matplotlib.pyplot as plt
from time_templates.datareader.get_data import fetch_MC_data_from_tree
from time_templates.templates.universality.S1000_model import set_Rmu_df
from time_templates.utilities.plot import plot_profile_1d, plot_hist
from time_templates.templates.universality.rho_model import set_total_signal_pred_df
from time_templates.templates.universality.names import DICT_COMP_COLORS, eMUON, eEM_HAD, eEM_PURE, eEM_MU, DICT_COMP_LABELS, DICT_COMP_MARKERS, DICT_COMP_SIGNALKEY

SAVE = True
BOOTSTRAPS = 0

if SAVE:
    plt.style.use('thesis')
    BOOTSTRAPS = 1000
# df = pd.read_pickle("/home/mart/auger/data/time_templates/df_rho_fitted.pl")
# # df = fetch_MC_data_from_tree(energy='19.5_20', primary='proton', dense=True, no_traces=True)

# df = df.query('WCDTotalSignal_pred > 5')
# df['MCCosTheta_sq'] = df['MCCosTheta']**2
thesis_fig_path = "/home/mart/auger/thesis/thesis_overleaf/figs/"

# %matplotlib notebook

In [ ]:
FORCE = False
dfs = []
for primary in ["proton", "iron"]:
    for energy in ["19.5_20"]:
        df = fetch_MC_data_from_tree(
            primary=primary,
            energy=energy,
            HIM="EPOS_LHC",
            key="new_UUB_SSD_rcola",
            no_traces=True,
            dense=True,
            univ_comp=True,
            force=FORCE
        )
        dfs.append(df)
        # Also non dense?
#         df = fetch_MC_data_from_tree(
#             primary=primary,
#             energy=energy,
#             HIM="EPOS_LHC",
#             key="new_UUB_SSD_rcola",
#             no_traces=True,
#             dense=False,
#             univ_comp=True,
#             force=FORCE
#         )
#         dfs.append(df)

df = pd.concat(dfs)

df = df.query(
    "LowGainSat == 0 & MCCosTheta > 0.6 & MCr < 2550 & MCr > 450 & MCDXstation > 10"
)  # & WCDTotalSignal > 10")

# TODO: Xmumax might not be OK
#    df["MCDXmumax"] = atmosphere.DX_at_station_isothermal(
#        df["MCr"], df["MCpsi"], df["MCTheta"], df["MCXmumax"]
#    )
set_Rmu_df(df)
set_total_signal_pred_df(df, Rmu=df["Rmu"].values)  # only for MC of course


df = df.query("WCDTotalSignal_pred > 5")
df['WCDTotalSignal_res'] = df['WCDTotalSignal']/df['WCDTotalSignal_pred']
df['MCCosTheta_sq'] = df['MCCosTheta']**2
df_p = df.query("primary == 'proton'")
df_Fe = df.query("primary == 'iron'")


In [ ]:
from time_templates.templates.event_templates import get_total_signals
from rho_model import get_rho_signal_comp

get_rho_signal_comp(np.deg2rad(40), 400, 1500, 0, eEM_PURE)

In [ ]:
df_p_ = df_p.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.5 & MClgE < 19.6')
r_bins = np.array([450, 550, 650, 900, 1100, 1300, 1500, 1700, 1900, 2050])
# r_bins = np.arange(450, 2200, 100)

f, ax = plt.subplots(1, figsize=(2.5, 2.5))#, figsize=(8, 6))
plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal'], bins=r_bins, ax=ax, color='k', marker='s', label='total signal', bootstraps=BOOTSTRAPS)
for comp, signalkey in DICT_COMP_SIGNALKEY.items():
    plot_profile_1d(df_p_['MCr'], df_p_[signalkey], bins=r_bins, ax=ax, color=DICT_COMP_COLORS[comp], marker=DICT_COMP_MARKERS[comp], label=f"${DICT_COMP_LABELS[comp]}$",
                   bootstraps=BOOTSTRAPS);

n = 100 
rs = np.linspace(500, 2050, n)
psis = np.ones(n) * np.pi/2 #np.arccos(df_p_['MCcospsi_round'].mean())
Rmu = df_p_['Rmu'].mean()
lgE = df_p_['MClgE'].mean()
Xmax = df_p_['MCXmax'].mean()
theta = df_p_['MCTheta'].mean()
Smu, Sem_mu, Sem_had, Sem_pure = get_total_signals(rs, psis, Rmu, lgE, Xmax, theta)

ax.plot(rs, Smu, color=DICT_COMP_COLORS[eMUON])
ax.plot(rs, Sem_mu, color=DICT_COMP_COLORS[eEM_MU])

ax.plot(rs, Sem_had, color=DICT_COMP_COLORS[eEM_HAD])

ax.plot(rs, Sem_pure, color=DICT_COMP_COLORS[eEM_PURE])

ax.plot(rs, Smu+Sem_mu+Sem_had+Sem_pure, 'k-')
# ax.legend(ncol=2, fontsize=9)
ax.set_yscale('log')
ax.set_xlabel('$r$ [m]')
ax.set_ylabel('Signal [VEM]')
ax.set_ylim([1e-1, None])
ax.axhline(5, ls=':', color='k')
if SAVE:
    plt.savefig(thesis_fig_path + 'example_r_signal_lgE19.55_theta38.pdf')

In [ ]:
bins = np.linspace(0.5, 1.5)
f, ax = plt.subplots(1, figsize=(2.5, 2.5))
plot_hist(df_p['WCDTotalSignal_res'], bins=bins, histtype='step', color='b', ax=ax, density=True)
plot_hist(df_Fe['WCDTotalSignal_res'], bins=bins, histtype='step', color='r', ax=ax, density=True)

ax.axvline(1, ls='--', color='k')
ax.set_xlabel(f'$S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}}$')
ax.set_ylabel('probability density')
ax.legend().remove()
if SAVE:
    plt.savefig(thesis_fig_path+'total_signal_all_stations_ratio_hist.pdf')

In [ ]:
f, ax = plt.subplots(1, figsize=(2.5, 2.5))
theta = np.deg2rad(38)
ct2 = np.cos(theta)**2
print(np.rad2deg(np.arccos(np.sqrt(ct2-0.05))), np.rad2deg(np.arccos(np.sqrt(ct2+0.05))))
df_p_ = df_p.query(f'MCCosTheta_sq > {ct2-0.05} & MCCosTheta_sq < {ct2+0.05}')
df_Fe_ = df_Fe.query(f'MCCosTheta_sq > {ct2-0.05} & MCCosTheta_sq < {ct2+0.05}')

# plot_profile_1d(df_p_['WCDTotalSignal_pred'], df_p_['WCDTotalSignal_res'], bins=np.logspace(np.log10(5), np.log10(2000), 21), ax=ax1,
#                 stat='mean', bootstraps=BOOTSTRAPS, color='b')
plot_profile_1d(df_p_['WCDTotalSignal_pred'], df_p_['WCDTotalSignal_res'], bins=np.logspace(np.log10(5), np.log10(2000), 21), ax=ax, 
                stat=np.std, bootstraps=BOOTSTRAPS, color='b')

# plot_profile_1d(df_Fe_['WCDTotalSignal_pred'], df_Fe_['WCDTotalSignal_res'], bins=np.logspace(np.log10(5), np.log10(2000), 21), ax=ax1,
#                 stat='mean', bootstraps=BOOTSTRAPS, color='r')
plot_profile_1d(df_Fe_['WCDTotalSignal_pred'], df_Fe_['WCDTotalSignal_res'], bins=np.logspace(np.log10(5), np.log10(2000), 21), ax=ax, 
                stat=np.std, bootstraps=BOOTSTRAPS, color='r')

# ax1.axhspan(0.95, 1.05, color='k', alpha=0.1)
# ax1.axhline(1, ls='--', color='k')
# ax1.set_ylim([0.8, 1.2])
# ax1.set_ylabel(f'$ \\langle S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}} \\rangle$')
# ax1.set_xlabel('$S_{\\rm total}$ [VEM]')
# ax1.set_xscale('log')

ax.set_xscale('log')
# ax2.set_yscale('log')
from time_templates.misc.sd_signal import wcd_total_signal_uncertainty
s = np.logspace(np.log10(5), np.log10(2000))

ax.plot(s, wcd_total_signal_uncertainty(s, theta)/s, 'k--', label='Uncertainty model data')
ax.set_ylabel(f'$ \\sigma \\left[S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}} \\right]$ [VEM]')
ax.set_xlabel('$S_{\\rm total}$ [VEM]')
ax.legend()
# plt.subplots_adjust(wspace=0.3)
if SAVE:
    plt.savefig(thesis_fig_path+"resolution_total_signal_vs_signal.pdf")

In [ ]:
bins = r_bins #np.array([450, 550, 650, 900, 1100, 1300, 1500, 1700, 1900, 2100, 2600]) #r_bins
df_p_ = df_p.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
df_Fe_ = df_Fe.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
f, ax = plt.subplots(1, figsize=(2.5, 2.5))

plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal_res'], bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='mean')
plot_profile_1d(df_Fe_['MCr'], df_Fe_['WCDTotalSignal_res'], bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='mean')
# plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
# plot_profile_1d(df_Fe_['MCr'], df_Fe_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
ax.axhline(1, ls='--', color='k')
ax.set_ylim([0.8, 1.2])
ax.set_ylabel(f'$\\langle S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}} \\rangle$')
ax.set_xlabel('$r$ [m]')
if SAVE:
    plt.savefig(thesis_fig_path+'profile_total_signal_ratio_vs_r.pdf')

In [ ]:
bins = r_bins #np.array([450, 550, 650, 900, 1100, 1300, 1500, 1700, 1900, 2100, 2600]) #r_bins
df_p_ = df_p.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
df_Fe_ = df_Fe.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
f, ax = plt.subplots(1, figsize=(2.5, 2.5))

plot_profile_1d(df_p_['MCr'], (df_p_['WCDTotalSignal'] - df_p_['WCDTotalSignal_pred'])/df_p_['WCDTotalSignal_err'], bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='mean')
plot_profile_1d(df_Fe_['MCr'], (df_Fe_['WCDTotalSignal'] - df_Fe_['WCDTotalSignal_pred'])/df_Fe_['WCDTotalSignal_err'], bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='mean')
# plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
# plot_profile_1d(df_Fe_['MCr'], df_Fe_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
ax.axhspan(-1, 1, color='k', alpha=0.1)
ax.axhline(0, ls='--', color='k')
ax.set_ylim([-3, 3])
ax.set_ylabel(f'$\\langle S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}} \\rangle$')
ax.set_xlabel('$r$ [m]')


In [ ]:
bins = r_bins #np.array([450, 550, 650, 900, 1100, 1300, 1500, 1700, 1900, 2100, 2600]) #r_bins
df_p_ = df_p.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
df_Fe_ = df_Fe.query('MCCosTheta_sq > 0.58 & MCCosTheta_sq < 0.67 & MClgE > 19.7 & MClgE < 19.8')
f, ax = plt.subplots(1, figsize=(2.5, 2.5))

plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal_res'], bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='mean')
plot_profile_1d(df_Fe_['MCr'], df_Fe_['WCDTotalSignal_res'], bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='mean')
# plot_profile_1d(df_p_['MCr'], df_p_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='b', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
# plot_profile_1d(df_Fe_['MCr'], df_Fe_['WCDTotalSignal_res']+1, bins=bins, ax=ax, color='r', bootstraps=BOOTSTRAPS, stat='median', mfc='none')
ax.axhspan(0.95, 1.05, color='k', alpha=0.1)
ax.axhline(1, ls='--', color='k')
ax.set_ylim([0.8, 1.2])
ax.set_ylabel(f'$\\langle S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}} \\rangle$')
ax.set_xlabel('$r$ [m]')

In [ ]:
plot_hist(df_Fe_.query('MCr > 1950 & MCr < 2050')['WCDTotalSignal_res'], bins=np.linspace(0, 2, 20), histtype='step', fit_norm=True)

In [ ]:
bins = np.array([-1.1, -0.9, -0.5, 0., 0.5, 0.9, 1.1]) 
f, ax = plt.subplots(1)

r = 1800
ct = 0.65

df_p_ = df_p.query(f'MCr == {r} & MCCosTheta > {ct-0.05} & MCCosTheta < {ct+0.05} & IsDense == 1')
df_Fe_ = df_Fe.query(f'MCr == {r} & MCCosTheta > {ct-0.05} & MCCosTheta < {ct+0.05} & IsDense == 1')

plot_profile_1d(df_p_['MCcospsi'], df_p_['WCDTotalSignal_res'], bins=bins, ax=ax)
plot_profile_1d(df_Fe_['MCcospsi'], df_Fe_['WCDTotalSignal_res'], bins=bins, ax=ax)

ax.axhspan(0.95, 1.05, color='k', alpha=0.1)

ax.axhline(1, ls='--', color='k')

ax.set_ylim([0.8, 1.2])
ax.set_ylabel(f'$S_{{\\rm total}} / \\widehat{{S}}_{{\\rm total}}$')
ax.set_xlabel('$\\cos{\\psi}$')